<a href="https://colab.research.google.com/github/Shota-8811396/output/blob/output/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E5%AE%9F%E8%A3%85%E2%91%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ネットワークの訓練

#### 1.データセットの準備

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'2.0.1+cu118'

In [2]:
# データセットの読み込み
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
x = breast_cancer['data']
t = breast_cancer['target']

In [3]:
x.shape, t.shape

((569, 30), (569,))

In [4]:
# PyTorch の Tensor 型へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [5]:
x.dtype, t.dtype

(torch.float32, torch.int64)

In [6]:
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [7]:
# 入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [8]:
# 各データセットのサンプル数を決定
# trian : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

# 乱数のシードを固定
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [9]:
len(train), len(val), len(test)

(341, 113, 115)

In [10]:
# バッチサイズの定義
batch_size = 10

In [11]:
# DataLoader を用意
# shuffle はデフォルトで False のため、訓練データのみ True(毎回ランダムに入れ替える) に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

#### 2.ネットワークの設計
今回の入力変数の数は M =30 で、これを悪性か良性の二値に分類する問題設定。

二値分類のため、ロジスティック回帰のように出力変数の数を 1 としても良いが、今後の多値分類にも対応できるよう、出力変数の数を 2 としたネットワークを定義する。

全結合層 fc1 と fc2 のノードの数を以下のように決定。

- fc1: input: 30 => output: 10
- fc2: input: 10 => output: 2

順伝播の計算の流れは以下のように行う

「線形変換 (fc1) => 非線形変換 (ReLU) => 線形変換 (fc2) => 非線形変換 (Softmax)」



In [12]:
x, t = train[0]

In [13]:
x.shape

torch.Size([30])

In [14]:
x, t

(tensor([1.3050e+01, 1.9310e+01, 8.2610e+01, 5.2720e+02, 8.0600e-02, 3.7890e-02,
         6.9200e-04, 4.1670e-03, 1.8190e-01, 5.5010e-02, 4.0400e-01, 1.2140e+00,
         2.5950e+00, 3.2960e+01, 7.4910e-03, 8.5930e-03, 6.9200e-04, 4.1670e-03,
         2.1900e-02, 2.9900e-03, 1.4230e+01, 2.2250e+01, 9.0240e+01, 6.2410e+02,
         1.0210e-01, 6.1910e-02, 1.8450e-03, 1.1110e-02, 2.4390e-01, 6.2890e-02]),
 tensor(1))

In [15]:
x.unsqueeze(0).shape

torch.Size([1, 30])

In [16]:
# 乱数シードを固定
torch.manual_seed(0)

# 全結合層の定義
fc1 = nn.Linear(30, 10)
fc2 = nn.Linear(10, 2)

順伝播
- 線形変換 fc1
- 非線形変換 ReLU
- 線形変換 fc2
- 非線形変換 Softmax

In [17]:
x.unsqueeze(0).shape

torch.Size([1, 30])

In [18]:
h = fc1(x.unsqueeze(0))
h

tensor([[  0.1830, -29.9828, -72.3173, 142.6534,  51.1358,  95.4793, -50.6195,
          87.3314,  94.4835,  10.6316]], grad_fn=<AddmmBackward0>)

In [19]:
h.shape

torch.Size([1, 10])

In [20]:
h = F.relu(h)
h

tensor([[  0.1830,   0.0000,   0.0000, 142.6534,  51.1358,  95.4793,   0.0000,
          87.3314,  94.4835,  10.6316]], grad_fn=<ReluBackward0>)

In [21]:
h = fc2(h)
h

tensor([[22.3666, 57.7020]], grad_fn=<AddmmBackward0>)

In [22]:
h = F.softmax(h)
h

<ipython-input-22-3e9aa2b0c6ea>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  h = F.softmax(h)


tensor([[4.5087e-16, 1.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [59]:
# クラスを定義
class Net(nn.Module):

    # 使用するオブジェクトを定義
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(30, 10)
        self.fc2 = nn.Linear(10, 2)


    # 順伝播の流れ
    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        # h = F.softmax(h) → softmax関数は記述しなくても良い
        return h

In [24]:
# 乱数シードを固定
torch.manual_seed(0)

# インスタンス化
net = Net()

In [25]:
# ネットワークの構造を確認
net

Net(
  (fc1): Linear(in_features=30, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [26]:
x.unsqueeze(0).shape

torch.Size([1, 30])

In [27]:
# net.forward(x.unsqueeze(0)) → .foward は省略可
net(x.unsqueeze(0))

tensor([[22.3666, 57.7020]], grad_fn=<AddmmBackward0>)

#### 3.損失関数

今回は２クラスであるが、多値分類となるようにネットワークを設計したため、損失関数として交差エントロピーを採用する。

交差エントロピー

$$
L = - \dfrac{1}{N} \sum_{n=1}^{N} \sum_{c=1}^{C} t_{nc} \log y_{nc}
$$

PyTorch では F.cross_entropy 

$$
u = \log \exp(u)
$$

で定義されており、Softmax関数は自動的に計算に組み込まれている。

そのため、ネットワークに softmax関数の記述はいらない。



##### 実際に u の値を計算してみる

In [28]:
import numpy as np

# u の値が正で大きい場合
u = 1000
print(f'u: {u}')
print(f'exp(u): {np.exp(u)}')
print(f'log exp(u): {np.log(np.exp(u))}')

# u の値が負で小さい場合
u = -1000
print(f'u: {u}')
print(f'exp(u): {np.exp(u)}')
print(f'log exp(u): {np.log(np.exp(u))}')

u: 1000
exp(u): inf
log exp(u): inf
u: -1000
exp(u): 0.0
log exp(u): -inf


<ipython-input-28-574f2fc193c8>:6: RuntimeWarning: overflow encountered in exp
  print(f'exp(u): {np.exp(u)}')
<ipython-input-28-574f2fc193c8>:7: RuntimeWarning: overflow encountered in exp
  print(f'log exp(u): {np.log(np.exp(u))}')
<ipython-input-28-574f2fc193c8>:13: RuntimeWarning: divide by zero encountered in log
  print(f'log exp(u): {np.log(np.exp(u))}')


この結果からわかる通り、値が大きすぎたり小さすぎる場合に計算の過程で情報が落ちてしまう影響により、本来の値と異なってしまう。

この問題を解決するために PyTorch では、交差エントロピーを計算する損失関数に softmax も一緒に計算するように設定してありる。

これにより情報落ちの問題を防ぎ、計算を安定させることができる。

ユーザー目線では、出力層の softmax 関数は定義しなくて良いと覚えておく。

#### 4.最適化手法

ネットワークの訓練にあたり使用する最適化手法を選択。

optimizer という変数名で定義することが一般的。

PyTorch では、torch.optim 内に様々な最適化手法が用意されており、今回は確率的勾配降下法 (SGD) を選択。

optimizer を定義する際に、引数としてネットワークのパラメータを渡す必要があり、パラメータの取得には net.parameters() を用いる。

もう１つの引数として学習係数 (lr: learning rate) も選択する。

In [29]:
# net.parameters()を展開して確認

# print(*net.parameters())

In [30]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

#### 5.ネットワークの流れを確認

訓練では、以下の 5 つの手順を繰り返す。

- ミニバッチ単位でサンプル X, t を抽出
- 現在のパラメータ W, b を利用して、順伝播で予測値 y を算出
- 目標値 t と予測値 y から損失関数 L を算出
- 誤差逆伝播法に基づいて各パラメータの勾配を算出
- 勾配の値に基づいて選択した最適化手法によりパラメータ W, bを更新


In [31]:
# for batch in train_loader:
#     print(batch)

In [32]:
# バッチサイズ分のサンプル数の抽出
batch = next(iter(train_loader))
batch

[tensor([[1.2770e+01, 2.9430e+01, 8.1350e+01, 5.0790e+02, 8.2760e-02, 4.2340e-02,
          1.9970e-02, 1.4990e-02, 1.5390e-01, 5.6370e-02, 2.4090e-01, 1.3670e+00,
          1.4770e+00, 1.8760e+01, 8.8350e-03, 1.2330e-02, 1.3280e-02, 9.3050e-03,
          1.8970e-02, 1.7260e-03, 1.3870e+01, 3.6000e+01, 8.8100e+01, 5.9470e+02,
          1.2340e-01, 1.0640e-01, 8.6530e-02, 6.4980e-02, 2.4070e-01, 6.4840e-02],
         [1.2460e+01, 2.4040e+01, 8.3970e+01, 4.7590e+02, 1.1860e-01, 2.3960e-01,
          2.2730e-01, 8.5430e-02, 2.0300e-01, 8.2430e-02, 2.9760e-01, 1.5990e+00,
          2.0390e+00, 2.3940e+01, 7.1490e-03, 7.2170e-02, 7.7430e-02, 1.4320e-02,
          1.7890e-02, 1.0080e-02, 1.5090e+01, 4.0680e+01, 9.7650e+01, 7.1140e+02,
          1.8530e-01, 1.0580e+00, 1.1050e+00, 2.2100e-01, 4.3660e-01, 2.0750e-01],
         [2.3290e+01, 2.6670e+01, 1.5890e+02, 1.6850e+03, 1.1410e-01, 2.0840e-01,
          3.5230e-01, 1.6200e-01, 2.2000e-01, 6.2290e-02, 5.5390e-01, 1.5600e+00,
          4.66

In [33]:
# 入力値と目標値に分割(step1)
x, t = batch

In [34]:
x.shape

torch.Size([10, 30])

In [35]:
# forward の計算(step2)
y = net(x)
y

tensor([[ 20.6692,  56.7171],
        [ 17.0223,  62.1575],
        [ 65.7915, 175.6876],
        [ 50.2580, 151.4434],
        [ 24.6620,  69.8544],
        [ 21.4863,  64.4037],
        [ 24.7339,  70.3620],
        [ 20.9930,  54.4346],
        [ 23.8985,  60.1556],
        [ 14.8905,  47.4721]], grad_fn=<AddmmBackward0>)

In [36]:
# 損失関数の計算(step3)
loss = F.cross_entropy(y, t)
loss

tensor(33.3991, grad_fn=<NllLossBackward0>)

In [37]:
# 全結合層 fc1 の重みに関する勾配
net.fc1.bias.grad

In [38]:
# 全結合層 fc1 のバイアスに関する勾配
net.fc1.bias.grad

In [39]:
# 勾配情報の初期化と勾配の算出(step4)
optimizer.zero_grad() # 初期化
loss.backward() # 算出

In [40]:
# 勾配の情報を用いたパラメータの更新(step5)
optimizer.step()

In [41]:
net.fc1.weight

Parameter containing:
tensor([[-2.4745e-02,  6.3585e-02, -3.0799e-01, -1.4166e+00, -7.0483e-02,
          4.8663e-02, -3.9600e-03,  1.4461e-01, -1.6507e-02,  4.8209e-02,
         -5.5844e-02, -3.8094e-02, -1.7996e-01, -1.9935e-01, -7.5272e-02,
          6.6719e-03,  7.2067e-02,  1.0952e-01, -1.2381e-01, -7.9515e-02,
          3.9725e-02,  1.0154e-01, -2.2250e-01, -1.5181e+00, -2.9663e-02,
          1.8435e-02,  1.6423e-01, -1.6967e-01, -1.1547e-01, -4.6403e-02],
        [-7.1167e-02,  1.5774e-01, -1.1834e-01, -8.4045e-02, -1.2755e-01,
         -1.7099e-01, -1.0658e-01,  1.5694e-01,  8.1468e-02,  8.8489e-02,
          9.6019e-03, -9.3603e-02,  3.0889e-02, -1.7047e-01, -1.3192e-01,
         -9.4122e-02,  1.1519e-01,  1.0705e-01, -8.0971e-02, -6.5877e-03,
          1.1677e-01,  1.8150e-01,  7.2460e-02,  2.4664e-02,  1.2241e-01,
         -1.0750e-01,  3.4022e-02, -1.4155e-01, -1.2654e-01, -9.4315e-02],
        [ 8.2610e-02,  7.3424e-02, -1.0815e-01,  5.5157e-02,  1.0023e-01,
         -2.30

In [42]:
# 演算に使用できる GPU の有無を確認
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [43]:
# 指定したデバイスへネットワークの転送
net.to(device)

Net(
  (fc1): Linear(in_features=30, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [44]:
x = x.to(device)
t = t.to(device)

In [45]:
y = net(x)
y

tensor([[ 148.3002, -284.6127],
        [ 163.9481, -312.1373],
        [ 493.4534, -945.2295],
        [ 430.1196, -824.0096],
        [ 185.5045, -355.8210],
        [ 170.7496, -327.5804],
        [ 187.2742, -359.2082],
        [ 145.2411, -278.7576],
        [ 156.5140, -300.3338],
        [ 121.6984, -232.1702]], device='cuda:0', grad_fn=<AddmmBackward0>)

#### 6.ネットワークを訓練

In [47]:
# エポック数
max_epoch = 1 # 1epoch = 34 iteration

In [48]:
# ネットワークの初期化
torch.manual_seed(0)
net = Net().to(device)

In [56]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.02)

In [60]:
# 訓練のループ
for epoch in range(max_epoch):

    for batch in train_loader:

        # step1
        x, t = batch
        x = x.to(device)
        t = t.to(device)

        # step2
        y = net(x)

        # step3
        loss = F.cross_entropy(y, t)
        print(f'loss: {loss}')

        # step4
        optimizer.zero_grad()
        loss.backward()

        # step5
        optimizer.step()

loss: 19.676408767700195
loss: 180.4086151123047
loss: 0.6949144601821899
loss: 0.6925337910652161
loss: 0.6931600570678711
loss: 0.6921578645706177
loss: 0.6931714415550232
loss: 0.6917929649353027
loss: 0.6949502229690552
loss: 0.6904767751693726
loss: 1.1271803379058838
loss: 0.690382182598114
loss: 0.6900392770767212
loss: 0.6824897527694702
loss: 0.6886609792709351
loss: 0.6729834675788879
loss: 0.6937034130096436
loss: 0.6870874166488647
loss: 0.7006877660751343
loss: 0.6807309985160828
loss: 0.6865910291671753
loss: 0.6787906289100647
loss: 0.669226348400116
loss: 0.7129955887794495
loss: 0.7025274634361267
loss: 0.6859496831893921
loss: 0.6690613031387329
loss: 0.6848255395889282
loss: 0.6651143431663513
loss: 0.6837738156318665
loss: 0.6724283695220947
loss: 0.6712482571601868
loss: 0.6700915694236755
loss: 0.6557782888412476


#### 7.評価指標の算出

In [62]:
x, t = next(iter(train_loader))

In [65]:
net.cpu()

Net(
  (fc1): Linear(in_features=30, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
)

In [66]:
y = net(x)
y

tensor([[-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958],
        [-0.3382, -0.1958]], grad_fn=<AddmmBackward0>)

In [67]:
t

tensor([0, 0, 0, 0, 1, 0, 1, 1, 0, 1])

In [71]:
y_label = torch.argmax(y, dim=1)
y_label

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [73]:
y_label == t

tensor([False, False, False, False,  True, False,  True,  True, False,  True])

In [75]:
(y_label == t).sum()

tensor(4)

In [76]:
len(t)

10

In [77]:
(y_label == t).sum() / len(t)

tensor(0.4000)

正解率を追加

In [79]:
# ネットワークの初期化
torch.manual_seed(0)
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.02)

In [80]:
# 訓練のループ
for epoch in range(max_epoch):

    for batch in train_loader:

        # step1
        x, t = batch
        x = x.to(device)
        t = t.to(device)

        # step2
        y = net(x)

        # 正解率(accuracy)を追加
        y_label = torch.argmax(y, dim=1)
        accuracy = (y_label == t).sum() / len(t)
        print(f'accuracy: {accuracy:2f}')

        # step3
        loss = F.cross_entropy(y, t)
        # print(f'loss: {loss}')

        # step4
        optimizer.zero_grad()
        loss.backward()

        # step5
        optimizer.step()

accuracy: 0.500000
accuracy: 0.300000
accuracy: 0.500000
accuracy: 0.000000
accuracy: 0.800000
accuracy: 0.600000
accuracy: 0.700000
accuracy: 0.600000
accuracy: 0.300000
accuracy: 0.600000
accuracy: 0.900000
accuracy: 0.800000
accuracy: 0.700000
accuracy: 0.500000
accuracy: 0.600000
accuracy: 0.400000
accuracy: 0.700000
accuracy: 0.800000
accuracy: 0.400000
accuracy: 0.700000
accuracy: 0.900000
accuracy: 0.500000
accuracy: 0.500000
accuracy: 0.400000
accuracy: 1.000000
accuracy: 0.500000
accuracy: 0.500000
accuracy: 0.400000
accuracy: 0.600000
accuracy: 0.600000
accuracy: 0.500000
accuracy: 0.600000
accuracy: 0.600000
accuracy: 0.700000


#### 8.訓練後の正解率を検証
with torch.no_grad

In [89]:
# val_lorder
def calc_accuracy(data_loader):

    with torch.no_grad():
        total = 0
        correct = 0

        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)

            y_label = torch.argmax(y, dim=1)
            total += len(t)
            correct += (y_label == t).sum()

        accuracy = correct / total

    return accuracy

In [93]:
# 検証データで確認
calc_accuracy(val_loader)

tensor(0.6637, device='cuda:0')

In [92]:
# テストデータで確認
calc_accuracy(test_loader)

tensor(0.6087, device='cuda:0')